In [40]:
import numpy as np
import cv2 as cv
import os
from sklearn.cluster import KMeans
from skimage.feature import hog
import matplotlib.pyplot as plt
import re

In [41]:
treinamentoDiretorio = "CAPTCHA-10k/treinamento"
validacaoDiretorio = "CAPTCHA-10k/validacao"

In [42]:
regioes = [35, 65, 95, 125, 155]
tamanhoDaRegiao = 2048

In [43]:
def padding_img(img: np.ndarray, multiply_of: int):
    nLines, nCols = img.shape
    
    pad_row = 0 if nLines % multiply_of == 0 else multiply_of - nLines % multiply_of
    pad_col = 0 if nCols % multiply_of == 0 else multiply_of - nCols % multiply_of

    pad_width = ((0, pad_row), (0, pad_col)) # 
    return np.pad(img, pad_width, mode='maximum') 

In [44]:
def returna_regioes(img, regioes):
    img_regioes = []

    regioes = [5] + regioes + [img.shape[1]]

    for i in range(len(regioes)-1):
        img_regioes.append(img[:, regioes[i]:regioes[i+1]])


    return img_regioes


In [45]:
def hogTreinamento(caminho: str):
    features = []
    
    files = os.listdir(treinamentoDiretorio)
    
    for file in files:
        filePath = os.path.join(treinamentoDiretorio, file)
        img = cv.imread(filePath, cv.IMREAD_GRAYSCALE)

        imgs = returna_regioes(img, regioes)

        for i in imgs:
            i = padding_img(i, 16)
            if i.size != tamanhoDaRegiao:
                print("Erro todas as regioes precisam ter o mesmo tamanho")
        
        for i in imgs:
            features.append(hog(i, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys',feature_vector=True, channel_axis=None))

    return features



In [46]:
features = hogTreinamento("qul")
features = np.asarray(features)

In [47]:
kmeans = KMeans(n_clusters=36, random_state=0, n_init="auto").fit(features)

In [48]:
descritores = np.empty((8000, 6))

In [49]:
def hog_classificacao(caminho: str, kmeans, descritor):
    
    files = os.listdir(caminho)
    
    for file in files:

        file_number = int(file[:-4])

        filePath = os.path.join(caminho, file)
        img = cv.imread(filePath, cv.IMREAD_GRAYSCALE)

        imgs = returna_regioes(img, regioes)

        for i in imgs:
            i = padding_img(i, 16)
            if i.size != tamanhoDaRegiao:
                print("Erro todas as regioes precisam ter o mesmo tamanho")
    
        
        features = []
        for i in imgs:
            features.append(hog(i, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys',feature_vector=True, channel_axis=None))

        descritor[file_number-1, :] = kmeans.predict(features)
    
    return descritor



In [50]:
tabelaClassificacao =  hog_classificacao(treinamentoDiretorio, kmeans, descritores)

In [51]:
alphaNumericos = {}

rotuloValidacaoCaminho = "Selecao/label_treino"

files = os.listdir(rotuloValidacaoCaminho)
    
for file in files:

    file_number = int(file[:-4])
    filePath = os.path.join(rotuloValidacaoCaminho, file)
    
    with open(filePath) as f:
        primeiraLinha = f.readline().rstrip()
    
    classificacao = tabelaClassificacao[file_number-1]
    
    for i in range(6):
        c = primeiraLinha[i]

        if c.isdigit():
            c = "N" + c
     
        alphaNumericos.setdefault(c, {}).setdefault(classificacao[i], 1)
        alphaNumericos[c][classificacao[i]] += 1


In [56]:
for key in alphaNumericos:
    print(key)
    frequencia = alphaNumericos[key]
    frequencia = dict(sorted(frequencia.items(), key=lambda item: item[1]))
    for x in list(frequencia)[-5:]:
        print ("key {}, value {} ".format(x,  frequencia[x]))
    print()
    print()
    

N
key 20.0, value 66 
key 7.0, value 79 
key 15.0, value 127 
key 16.0, value 152 
key 17.0, value 762 


N0
key 11.0, value 72 
key 33.0, value 113 
key 23.0, value 126 
key 13.0, value 200 
key 25.0, value 322 


N2
key 35.0, value 89 
key 10.0, value 96 
key 5.0, value 101 
key 2.0, value 330 
key 29.0, value 535 


S
key 9.0, value 70 
key 10.0, value 82 
key 5.0, value 167 
key 21.0, value 281 
key 35.0, value 585 


H
key 15.0, value 110 
key 7.0, value 149 
key 28.0, value 180 
key 30.0, value 211 
key 18.0, value 288 


T
key 7.0, value 55 
key 28.0, value 56 
key 6.0, value 111 
key 27.0, value 221 
key 8.0, value 641 


X
key 16.0, value 16 
key 2.0, value 23 
key 3.0, value 36 
key 22.0, value 169 
key 26.0, value 1029 


G
key 34.0, value 156 
key 13.0, value 163 
key 33.0, value 207 
key 11.0, value 269 
key 1.0, value 291 


N6
key 33.0, value 117 
key 25.0, value 172 
key 13.0, value 223 
key 34.0, value 249 
key 11.0, value 271 


W
key 3.0, value 17 
key 16.0, value 18

In [62]:
labels = {}
cluster_label = np.array(36, dtype=bool)

In [63]:
taxas = np.linspace(.9, .1, 9)
diff = {}
for key in alphaNumericos:
    frequencia = list(alphaNumericos[key])
    diff[key] = 1 - alphaNumericos[key][frequencia[-2]] / alphaNumericos[key][frequencia[-1]] 

In [64]:
for taxa in taxas:
    for key in diff:
        if diff[key] > taxa:
            cluster = 
            labels[key] = list(alphaNumericos[key])[-1]

            

In [61]:
labels

{'N4': 0.0,
 'A': 31.0,
 'N': 17.0,
 'X': 26.0,
 'Y': 16.0,
 'N7': 2.0,
 'J': 19.0,
 'U': 14.0,
 'M': 17.0,
 'V': 4.0,
 'T': 8.0,
 'N1': 6.0,
 'Z': 29.0,
 'L': 14.0,
 'S': 35.0,
 'W': 4.0,
 'K': 22.0,
 'N5': 21.0,
 'N0': 25.0,
 'N2': 29.0,
 'F': 30.0,
 'N3': 35.0,
 'N9': 10.0,
 'E': 12.0,
 '?': 32.0,
 'H': 18.0,
 'R': 30.0,
 'P': 30.0,
 'Q': 1.0,
 'I': 27.0,
 'N8': 5.0,
 'B': 12.0,
 'D': 20.0}

In [ ]:
def hog2(img):
    # Calcula a derivada em x
    dxFilter = np.array((-1, 0, 1))
    
    dxImage = np.empty((img.shape[0]-2, img.shape[1]-2))
    nLines, nCols = dxImage.shape

    for i in range(nLines):
        for j in range(nCols):
            dxImage[i, j] = (img[i+1, j:j+3] * dxFilter).sum()
    
    
    # Calcula a derivada em y
    dyImage = np.empty((img.shape[0]-2, img.shape[1]-2))
    nLines, nCols = dyImage.shape

    for j in range(nCols):
        for i in range(nLines):
            dyImage[i, j] = (img[i:i+3, j+1] * dyFilter).sum()
    dyFilter = dxFilter.T


    pass

In [ ]:
img = cv.imread('CAPTCHA-10k/treinamento/000001.jpg', cv.IMREAD_GRAYSCALE)

In [ ]:
dxFilter = np.array((-1, 0, 1))
dyFilter = dxFilter.T

In [ ]:
dxImage = np.empty((img.shape[0]-2, img.shape[1]-2))
nLines, nCols = dxImage.shape

for i in range(nLines):
    for j in range(nCols):
        dxImage[i, j] = (img[i+1, j:j+3] * dxFilter).sum()



In [ ]:
dyImage = np.empty((img.shape[0]-2, img.shape[1]-2))
nLines, nCols = dyImage.shape

for j in range(nCols):
    for i in range(nLines):
        dyImage[i, j] = (img[i:i+3, j+1] * dyFilter).sum()


In [ ]:
newImage = np.empty_like(dxImage)
nLines, nCols = newImage.shape

if nLines % 8 != 0 and nCols % 8 != 0:
    newImage = padding_img(newImage)

In [ ]:
nLines, nCols = newImage.shape
if nLines % 8 != 0 and nCols % 8 != 0:
    print("Deu muita merda")

In [ ]:
descritor = np.array(newImage.size / 4 * 9)

In [ ]:
def calcula_2x2(imgBlock, dxBlock, dyBlock):
    magnitude = np.sqrt(dxBlock**2 + dyBlock**2)
    orientacao = np.arctan2(dxBlock, dyBlock)
    print(orientacao)
    exit(2)


In [ ]:
def calcula_8x8(imgBlock, dxBlock, dyBlock):
    for i in range(4):
        for j in range(4):
            calcula_2x2(img[i:i+2, j+2], dxBlock[i:i+2, j:j+2], dyBlock[i:i+2, j:j+2])
    return descritor


In [ ]:
for i in range(nLines // 8):
    for j in range(nCols // 8):
        calcula_8x8(newImage[i:i+8, j:j+8], dxImage[i:i+8, j:j+8], dyImage[i:i+8, j:j+8])

[[-0.10487694  0.04298426]
 [-0.24590117 -0.71386931]]
[[ 0.04298426  0.        ]
 [-0.71386931 -0.88312507]]
[[ 0.         -0.19270076]
 [-0.88312507 -0.35038235]]
[[-0.19270076  2.49809154]
 [-0.35038235 -0.14723192]]
[[-0.24590117 -0.71386931]
 [-1.4986327  -1.69887917]]
[[-0.71386931 -0.88312507]
 [-1.69887917  1.09168603]]
[[-0.88312507 -0.35038235]
 [ 1.09168603  0.50377901]]
[[-0.35038235 -0.14723192]
 [ 0.50377901 -0.35004489]]
[[-1.4986327  -1.69887917]
 [-1.57079633 -1.08990905]]
[[-1.69887917  1.09168603]
 [-1.08990905  0.74448533]]
[[1.09168603 0.50377901]
 [0.74448533 1.52949751]]
[[ 0.50377901 -0.35004489]
 [ 1.52949751  0.05258306]]
[[-1.57079633 -1.08990905]
 [-1.27535549 -0.46364761]]
[[-1.08990905  0.74448533]
 [-0.46364761  0.18280988]]
[[0.74448533 1.52949751]
 [0.18280988 1.5475447 ]]
[[1.52949751 0.05258306]
 [1.5475447  0.32175055]]
[[ 0.04298426  0.        ]
 [-0.71386931 -0.88312507]]
[[ 0.         -0.19270076]
 [-0.88312507 -0.35038235]]
[[-0.19270076  2.49809